<h1>Questions to be handed in on integration:</h1>

<p>To get started, we load the <code>Gadfly</code> backend for <code>Plots</code>, so that we can make plots; load the <code>Roots</code> package for its <code>D</code> function; and the <code>SymPy</code> package:</p>

In [ ]:
using Plots
gadfly()
using Roots			# for D and fzero
using SymPy

<h3>Quick background</h3>

<p>Read more about this material here: <a href="http://mth229.github.io/integration.html">integration</a>.</p>

<p>For the impatient, in many cases, the task of evaluating a definite integral is made easy by the fundamental theorem of calculus which says that for a continuous function $f$ the following holds for any antiderivate, $F$, of $f$:</p>


$$
\int_a^b f(x) dx = F(b) - F(a).
$$


<p>That is the definite integral is found by evaluating a related function at the endpoints, $a$ and $b$.</p>

<p>The <code>SymPy</code> package can compute many antiderivatives using a version of the <a href="http://en.wikipedia.org/wiki/Risch\_algorithm">Risch algorithm</a> that works for <a href="http://en.wikipedia.org/wiki/Elementary\_function">elementary functions</a>. <code>SymPy</code>'s <code>integrate</code> function can be used to find an indefinite integral:</p>

In [ ]:
f(x) = x^2
integrate(f)

<p>Or a definite integral:</p>

In [ ]:
integrate(f, 0, 1)		# returns a "symbolic" number

<p>However, this only works <em>if</em> there is a known antiderivative $F(x)$ $-$ which is not always the case. If not, what to do?</p>

<p>In this case, we can appeal to the definition of the definite integral. For continuous, non-negative $f(x)$, the definite integral is the area under the graph of $f$ over the interval $[a,b]$. For possibly negative functions, the indefinite integral is found by the <em>signed</em> area under $f$.  This area can be directly <em>approximated</em> using Riemann sums, or some other approximation scheme.</p>

<p>The Riemann approximation can help. The following pattern will compute a Riemann sum using right-hand endpoints:</p>

In [ ]:
f(x) = x^2
a, b, n = 0, 1, 5		# 5 partitions of [0,1] requested
delta = (b - a)/n		# size of partition
xs = a + (1:n) * delta	
fxs = [f(x) for x in xs]
sum(fxs * delta)		# a new function `sum` to add up values in a container

0.44000000000000006

<p>That value isn't very close to $1/3$. But we only took $n=5$ rectangles $-$ clearly there will be some error. Bigger $n$s mean better approximations:</p>

In [ ]:
f(x) = x^2
a, b, n = 0, 1, 50_000		# 50,000 partitions of [0,1] requested
delta = (b - a)/n		
xs = a + (1:n) * delta	
fxs = [f(x) for x in xs]
sum(fxs * delta)

0.3333433334

<p>Note that only the first two lines needed changing to adjust to a new problem. As the pattern is similar, it is fairly easy to wrap the computations in a function for convenience. We borrow this more elaborate one from the notes that works for some other methods beside the default right-Riemann sum:</p>

In [ ]:
function riemann(f::Function, a::Real, b::Real, n::Int; method="right")
  if method == "right"
     meth(f,l,r) = f(r) * (r-l)
  elseif method == "left"
     meth(f,l,r) = f(l) * (r-l)
  elseif method == "trapezoid"
     meth(f,l,r) = (1/2) * (f(l) + f(r)) * (r-l)
  elseif method == "simpsons"
     meth(f,l,r) = (1/6) * (f(l) + 4*(f((l+r)/2)) + f(r)) * (r-l)
  end

  xs = a + (0:n) * (b-a)/n
  as = [meth(f, l, r) for (l,r) in zip(xs[1:end-1], xs[2:end])]
  sum(as)
end

riemann (generic function with 1 method)

<p>The Riemann sum is very slow to converge here. There are faster algorithms both mathematically and computationally. We will discuss two: the trapezoid rule, which replaces rectangles with trapezoids; and Simpson's rule which is a quadratic approximation.</p>

In [ ]:
f(x) = x^2
riemann(f, 0, 1, 1000, method="trapezoid"), riemann(f, 0, 1, 1000, method="simpsons")

(0.33333350000000006,0.3333333333333337)

<p>Base <code>julia</code> provides the <code>quadgk</code> function which uses a different approach altogether. It is used quite easily:</p>

In [ ]:
f(x) = x^2
ans, err = quadgk(f, 0, 1)

(0.3333333333333333,5.551115123125783e-17)

<p>The <code>quadgk</code> function returns two values, an answer and an estimated maximum possible error.  The ans is the first number, clearly it is $1/3$, and the estimated maximum error is the second. In this case it is small ($10^{-17}$) $-$ basically 0.</p>

<h3>Questions</h3>

<ul>
<li>Let $g(x) = x^4 + 10x^2 - 60x + 71$. Find the integral $\int_0^1   g(x) dx$ by hand by finding an antiderivative and then using the fundamental theorem of calculus.</li>
</ul>

<ul>
<li>For $f(x) = x/\sqrt{g(x)}$ (for $g(x)$ from the last problem) estimate the following using 1000 Riemann sums:</li>
</ul>


$$
\int_0^1 f(x) dx
$$


<ul>
<li>Let $f(x) = \sin(\pi x^2)$. Estimate $\int_0^1 f(x) dx$ using 20 right-Riemann sums</li>
</ul>

<ul>
<li>For the same $f(x)$, compare your estimate with 20 Riemann sums to   that with 20,000 Riemann sums. How many digits after the decimal   point do they agree?</li>
</ul>

<h4>Left Riemann</h4>

<p>The left Riemann sum uses left-hand endpoints, not right-hand ones. </p>

<ul>
<li>For $f(x) = e^{x}$ use the left Riemann sum with $n=10,000$ to estimate $\int_0^1 f(x) dx$.</li>
</ul>

<ul>
<li>The left and right Riemann sums for an increasing function are also   lower and upper bounds for the answer. Find the difference between   the left and right Riemann sum for $\int_0^1 e^x dx$ when   $n=10,000$. (Use your last answer.) What is the approximate value   $1/100$, $1/1000$, $1/10000$, or $1/100000$?</li>
</ul>

<h4>Trapezoid, Simpson's</h4>

<ul>
<li>The answer to $\int_0^1 e^x dx$ is simply $e^1 - e^0$ =   $e-1$. Compare the error (in absolute value) of the trapezoid method when $n=10,000$.</li>
</ul>

<ul>
<li>The answer to $\int_0^1 e^x dx$ is simply $e^1 - e^0$ =   $e-1$. Compare the error of the Simpson's method when $n=10,000$.</li>
</ul>

<p>(The error for Riemann sums is "like" $1/n$, the error for trapezoid sums is like $1/n^2$, and for Simpson's rule the error is like $1/n^4$.)</p>

<h2>quadgk</h2>

<ul>
<li>Use <code>quadgk</code> to find $\int_{-3}^{3} (1 + x^2)^{-1} dx$. What is the answer? What is the estimated maximum error?</li>
</ul>

<p>The answer is:</p>

<p>The error is about</p>

<ul>
<li>Use <code>quadgk</code> to find the integral of $e^{-|x|}$ over $[-1,1]$.</li>
</ul>

<ul>
<li>The integral of $\sqrt{1-x^4}$ over $[-1,1]$ can not be found with the Fundamental Theorem of Calculus using an elementary function for an antiderivative. What is the <em>approximate</em> value?</li>
</ul>

<ul>
<li>The integral of $f(x) = \log(log(x))$ over $[e,e^2]$ can not be   found with the Fundamental Theorem of Calculus using an elementary   function for an antiderivative. What is the <em>approximate</em> value?</li>
</ul>

<p>The graph of $f(x)$ over the interval $[e, e^2]$ makes clear that the triangle formed by the line connecting $(e, f(e))$ and $(e^2, f(e^2))$, the $x$ axis, and the line $x=f(e^2)$ will form a lower bound for the area under $f$. What is the error in this approximation? (Where error = answer $-$ approximation.)</p>

<ul>
<li>A formula to compute the length of a the graph of the function $f(x)$ from $a$ to $b$ is given by the formula:</li>
</ul>


$$
\int_a^b \sqrt{1 + f'(x)^2} dx
$$


<p>Use this formula when $f(x) = \sin(x)$ and the interval is $[0,\pi]$. What is the answer?</p>

<p>Repeat, when the function is $f(x) = x^x$ over $(0, 3)$:</p>

<ul>
<li>Compute the area between the intersection points of the two curves $f(x) = x$ and $g(x) = x^3$ by taking the difference between two definite integrals.</li>
</ul>

<h2>Applications</h2>

<p>We discuss an application of the integral to finding the volumes $-$ not just areas.</p>

<p>A <em>solid of revolution</em> is a figure with rotational symmetry around some axis, such as a soda can, a snow cone, a red solo cup, and other common objects. A formula for the volume of an object with rotational symmetry can be written in terms of an integral based on a function, $r(h)$, which specifies the radius for various values of $h$.</p>

<blockquote>
<p>If the radius as a function of height is given by $r(h)$, the the volume is $\int_a^b \pi r(h)^2 dh$.</p>
</blockquote>

<p>So for example, a baseball has a overall diameter of $2\cdot 37$mm, but if we place the center at the origin, its rotational radious is given by $r(h) = (37^2 - h^2)^{1/2}$ for $-37 \leq h \leq 37$. The volume in mm$^3$ is given by:</p>

In [ ]:
r(h) = (37^2 - h^2)^(1/2)
v(h) = pi * r(h)^2
quadgk(v, -37, 37)

(212174.79024304505,2.9103830456733704e-11)

<h3>Glass half full</h3>

<ul>
<li>A glass is formed as a volume of revolution with radius as a   function of height given the equation $r(h) = 2 + (h/20)^4$. The   volume as a function of height $b$ is given by $V(b) = \int_0^b \pi   r(h)^2 dh$. Find $V(25)$. Show your work.</li>
</ul>

<ul>
<li>Find a value of $b$ so that $V(b) = 455$.</li>
</ul>

<ul>
<li>Now find a value of $b$ for which $V(b) = 455/2$. This height will   have half the volume as the height just found. Compare the two   values. Is the ratio of smallest to largest 1/2, more than 1/2 or   less?</li>
</ul>